In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_10%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,NaN,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,NaN,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,NaN,0.0
744,NaN,2.0,NaN,52.0,0.0
745,NaN,NaN,NaN,62.0,0.0
746,39.0,1.0,250.0,NaN,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              107
Frequency (times)                              78
Monetary (c.c. blood)                          94
Time (months)                                  95
whether he/she donated blood in March 2007      0
dtype: int64

In [6]:
df.isnull().sum().sum()

374

In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              14.304813
Frequency (times)                             10.427807
Monetary (c.c. blood)                         12.566845
Time (months)                                 12.700535
whether he/she donated blood in March 2007     0.000000
dtype: float64

In [8]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 374
persentase : 10.0


In [9]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [10]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,9)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
    
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [11]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [12]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,0.0,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,0.0,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,0.0,0.0
744,0.0,2.0,0.0,52.0,0.0
745,0.0,0.0,0.0,62.0,0.0
746,39.0,1.0,250.0,0.0,0.0


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)
dummy

Kolom : 1
[0.0, 1.0, 2.0, 2.0, 2.0, 4.0, 12.0, 14.0, 21.0]
min :  0.0
max :  21.0
mean :  6.444444444444445
fuzzy :  [(2.0, 0.3103448275862069), (4.0, 0.6206896551724138), (1.0, 0.15517241379310345), (2.0, 0.3103448275862069), (14.0, 0.48091603053435117), (21.0, 0.0), (12.0, 0.6183206106870229), (2.0, 0.3103448275862069), (0.0, 0.0)]
fuzzy :  None
pembilang :  18.65267175572519
penyebut :  2.806133192945512
rata-rata tertimbang :  6.647108484592655
 
[0.0, 2.0, 4.0, 4.0, 4.0, 9.0, 9.0, 11.0, 11.0]
min :  0.0
max :  11.0
mean :  6.0
fuzzy :  [(4.0, 0.6666666666666666), (0.0, 0.0), (4.0, 0.6666666666666666), (9.0, 0.4), (11.0, 0.0), (9.0, 0.4), (11.0, 0.0), (2.0, 0.3333333333333333), (4.0, 0.6666666666666666)]
fuzzy :  None
pembilang :  15.866666666666665
penyebut :  3.1333333333333333
rata-rata tertimbang :  5.0638297872340425
 
[1.0, 2.0, 2.0, 3.0, 4.0, 5.0, 6.0, 11.0, 11.0]
min :  1.0
max :  11.0
mean :  5.0
fuzzy :  [(11.0, 0.0), (1.0, 0.0), (5.0, 1.0), (6.0, 0.8333333333333334), (11

[0.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0, 14.0, 14.0]
min :  0.0
max :  14.0
mean :  5.222222222222222
fuzzy :  [(0.0, 0.0), (2.0, 0.3829787234042553), (2.0, 0.3829787234042553), (3.0, 0.5744680851063829), (4.0, 0.7659574468085106), (4.0, 0.7659574468085106), (4.0, 0.7659574468085106), (14.0, 0.0), (14.0, 0.0)]
fuzzy :  None
pembilang :  12.446808510638299
penyebut :  3.6382978723404253
rata-rata tertimbang :  3.421052631578948
 
[0.0, 2.0, 2.0, 4.0, 4.0, 11.0, 11.0, 13.0, 16.0]
min :  0.0
max :  16.0
mean :  7.0
fuzzy :  [(4.0, 0.5714285714285714), (4.0, 0.5714285714285714), (11.0, 0.5555555555555556), (11.0, 0.5555555555555556), (13.0, 0.3333333333333333), (2.0, 0.2857142857142857), (0.0, 0.0), (16.0, 0.0), (2.0, 0.2857142857142857)]
fuzzy :  None
pembilang :  22.269841269841272
penyebut :  3.1587301587301586
rata-rata tertimbang :  7.050251256281408
 
[2.0, 2.0, 4.0, 4.0, 7.050251256281408, 11.0, 11.0, 13.0, 16.0]
min :  2.0
max :  16.0
mean :  7.783361250697933
fuzzy :  [(11.0, 0.6085213

[0.0, 0.0, 2.0, 11.0, 11.0, 18.0, 21.0, 23.0, 23.0]
min :  0.0
max :  23.0
mean :  12.11111111111111
fuzzy :  [(0.0, 0.0), (2.0, 0.1651376146788991), (18.0, 0.45918367346938777), (21.0, 0.18367346938775508), (23.0, 0.0), (23.0, 0.0), (0.0, 0.0), (11.0, 0.908256880733945), (11.0, 0.908256880733945)]
fuzzy :  None
pembilang :  32.434375585096426
penyebut :  2.624508519003932
rata-rata tertimbang :  12.358266452648476
 
[0.0, 9.0, 11.0, 11.0, 11.0, 11.0, 14.0, 14.0, 16.0]
min :  0.0
max :  16.0
mean :  10.777777777777779
fuzzy :  [(0.0, 0.0), (11.0, 0.9574468085106385), (9.0, 0.8350515463917525), (14.0, 0.3829787234042554), (11.0, 0.9574468085106385), (11.0, 0.9574468085106385), (11.0, 0.9574468085106385), (16.0, 0.0), (14.0, 0.3829787234042554)]
fuzzy :  None
pembilang :  60.366527747313015
penyebut :  5.430796227242817
rata-rata tertimbang :  11.115594329334787
 
[0.0, 0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0]
min :  0.0
max :  5.0
mean :  3.2222222222222223
fuzzy :  [(0.0, 0.0), (0.0, 0.

[0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 14.0, 16.0, 21.0]
min :  0.0
max :  21.0
mean :  7.0
fuzzy :  [(0.0, 0.0), (4.0, 0.5714285714285714), (16.0, 0.35714285714285715), (14.0, 0.5), (0.0, 0.0), (21.0, 0.0), (0.0, 0.0), (4.0, 0.5714285714285714), (4.0, 0.5714285714285714)]
fuzzy :  None
pembilang :  19.57142857142857
penyebut :  2.571428571428571
rata-rata tertimbang :  7.611111111111111
 
[0.0, 0.0, 2.0, 9.0, 11.0, 11.0, 12.358266452648476, 16.0, 16.0]
min :  0.0
max :  16.0
mean :  8.595362939183165
fuzzy :  [(0.0, 0.0), (11.0, 0.6752525422830691), (0.0, 0.0), (9.0, 0.9453535591962966), (11.0, 0.6752525422830691), (16.0, 0.0), (2.0, 0.23268360093123236), (12.358266452648476, 0.49181796723333127), (16.0, 0.0)]
fuzzy :  None
pembilang :  29.907122650126098
penyebut :  3.0203602119269988
rata-rata tertimbang :  9.901839698466054
 
[0.0, 0.0, 1.0, 4.0, 5.0, 7.611686768896811, 11.0, 16.0, 20.0]
min :  0.0
max :  20.0
mean :  7.179076307655202
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (4.0, 0.557174743460

[0.0, 2.0, 4.0, 4.0, 4.0, 5.0, 11.0, 14.0, 21.0]
min :  0.0
max :  21.0
mean :  7.222222222222222
fuzzy :  [(0.0, 0.0), (4.0, 0.5538461538461539), (11.0, 0.7258064516129031), (14.0, 0.5080645161290323), (4.0, 0.5538461538461539), (5.0, 0.6923076923076923), (4.0, 0.5538461538461539), (2.0, 0.27692307692307694), (21.0, 0.0)]
fuzzy :  None
pembilang :  25.75831265508685
penyebut :  3.8646401985111662
rata-rata tertimbang :  6.6651256862178565
 
[0.0, 2.0, 2.0, 2.358974358974359, 4.0, 4.0, 5.0, 11.0, 11.0]
min :  0.0
max :  11.0
mean :  4.595441595441596
fuzzy :  [(0.0, 0.0), (2.0, 0.43521388716676995), (11.0, 0.0), (2.0, 0.43521388716676995), (5.0, 0.9368327402135231), (11.0, 0.0), (4.0, 0.8704277743335399), (2.358974358974359, 0.5133292002479851), (4.0, 0.8704277743335399)]
fuzzy :  None
pembilang :  14.599371865500824
penyebut :  4.061445263462128
rata-rata tertimbang :  3.5946247994133436
 
[0.0, 0.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0, 4.0]
min :  0.0
max :  4.0
mean :  2.4444444444444446
f

...

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888888888888
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  8.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.8888888

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.4444444444444444
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.5555555555555556
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.6666666

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.7777777777777778
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  7.0
penyebut :  9.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  5
min :  0.0
max :  1.0
mean :  0.4444444

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.000000,50.000000,7224.676478,98.000000,1.0
1,6.647108,13.000000,3250.000000,28.000000,1.0
2,1.000000,16.000000,4000.000000,71.785714,1.0
3,2.000000,20.000000,5000.000000,45.000000,1.0
4,1.000000,24.000000,6000.000000,77.000000,1.0
...,...,...,...,...,...
743,23.000000,2.000000,500.000000,24.460213,1.0
744,11.662197,2.000000,659.831598,52.000000,1.0
745,6.019163,7.107344,1778.181665,62.000000,1.0
746,39.000000,1.000000,250.000000,23.000000,1.0


In [15]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.0
mean,9.265864,5.525779,1349.964123,34.073394,1.0
std,7.746649,5.840019,1389.702306,24.180637,0.0
min,1.000000,1.000000,250.000000,2.000000,1.0
25%,3.415844,2.000000,500.000000,16.000000,1.0
50%,7.608838,4.000000,1000.000000,28.000000,1.0
75%,14.000000,7.000000,1750.000000,49.000000,1.0
max,74.000000,50.000000,11500.000000,98.000000,1.0


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [17]:
x = dummy.values
y = df.iloc[:, -1].values

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=94)

In [19]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [20]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[125  25]
 [ 22  15]]
0.7486631016042781


In [21]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[140  10]
 [ 32   5]]
0.7754010695187166


In [22]:
classifier = MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.1, max_iter=100)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[150   0]
 [ 37   0]]
0.8021390374331551
